In [ ]:
import requests
import gzip
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/BRCA/"

Download methyl, rna and clinical data

In [ ]:
def download_xena_file(url, filename):
    print(f"Downloading {filename}...")
    response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(response.content)
    print("Done!")

In [ ]:
# Download RNA-seq, Methylation, and PAM50
download_xena_file("https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.BRCA.sampleMap/HiSeqV2.gz", "brca_rna.gz")
download_xena_file("https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.BRCA.sampleMap/HumanMethylation450.gz", "brca_methyl.gz")
download_xena_file("https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.BRCA.sampleMap/BRCA_clinicalMatrix", "brca_clinical.tsv")

Done!
Done!
Done!


In [ ]:
methyl = pd.read_csv("brca_methyl.gz", sep="\t", index_col=0)
clinical = pd.read_csv("brca_clinical.tsv", sep="\t", index_col=0)

In [ ]:
methyl.shape, clinical.shape

((485577, 888), (1247, 193))

In [ ]:
# filter for subcategory type
for i in clinical.columns:
    if 'pam' in i.lower():
        print(i)

print(clinical['PAM50Call_RNAseq'].unique())

valid_samples = clinical[clinical['PAM50Call_RNAseq'].isin(['LumA','LumB','Her2','Basal'])].index
valid_samples_in_methyl = [s for s in valid_samples if s in methyl.columns]
methyl = methyl[valid_samples_in_methyl]
print(methyl.shape)

# Save cleaned data
methyl.to_csv(path+"cleaned_methyl.csv")


Integrated_Clusters_with_PAM50__nature2012
PAM50Call_RNAseq
PAM50_mRNA_nature2012
[nan 'Normal' 'LumA' 'LumB' 'Basal' 'Her2']
(485577, 533)


In [ ]:
# pull samples’ PAM50 label
labels = clinical.loc[valid_samples, 'PAM50Call_RNAseq']

# save labels
labels.to_csv(path+"brca_labels.csv", header=['PAM50'])

Feature Engineering

In [ ]:
# 1. load data
methyl_t = methyl.T
methyl_t = methyl_t[[col for col in methyl_t if col.startswith('cg')]]
labels_t = labels
print(methyl_t.shape, labels_t.shape)

# 2. align samples
common_samples = list(set(methyl_t.index) & set(labels_t.index))
print(f"total {len(common_samples)} samples aligned")

# 3. save data
X_train = methyl_t.loc[common_samples]
##X_train = X_train.fillna(X_train.mean())
X_train = X_train.fillna(0)
y_train = labels_t.loc[common_samples]
print(X_train.shape, y_train.shape)


(533, 482421) (837,)
total 533 samples aligned
(533, 482421) (533,)


Select top 50k methyl features

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=50000) # Select top 50k features
selector.fit(X_train, y_train)
selected_features_mask = selector.get_support()
selected_column_names = X_train.columns[selected_features_mask].tolist()
methyl_highvar_cg = methyl_t[selected_column_names]

# Save output
methyl_highvar_cg.to_csv(path+"methyl_top50k_cg_anova.csv")

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [     5     14     16 ... 482413 482414 482416] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
methyl_highvar_cg.shape

(533, 50000)

Extract all protein-coding gene names from the Gencode annotation file

In [ ]:
#import requests

url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz"
output_file = path+"gencode.v44.annotation.gtf.gz"

print("Start streaming download GENCODE v44 annotation file...")

response = requests.get(url, stream=True)
with open(output_file, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

print("✅ Download complete：", output_file)


Start streaming download GENCODE v44 annotation file...
✅ Download complete： /content/drive/MyDrive/Colab Notebooks/BRCA/gencode.v44.annotation.gtf.gz


In [ ]:
#import gzip
symbols = set()
with gzip.open(path+"gencode.v44.annotation.gtf.gz","rt") as f:
    for line in f:
        if line.startswith("#"): continue
        fields = line.split('\t')
        if fields[2] != "gene": continue
        info = fields[8]
        if 'gene_type "protein_coding"' in info:
            for attr in info.split(';'):
                attr = attr.strip()
                if attr.startswith("gene_name"):
                    symbols.add(attr.split('"')[1])
with open(path+"protein_coding_symbols.txt","w") as out:
    out.write("\n".join(sorted(symbols)))

print("# of gene symbols: ", len(symbols))
pc_genes = symbols

# of gene symbols:  20020


In [ ]:
# Load RNA-seq expression data and filter by protein-coding genes
rna = pd.read_csv("brca_rna.gz", sep="\t", index_col=0)
print(rna.shape)
rna_pc = rna[rna.index.isin(pc_genes)]
rna_pc.to_csv(path+"rna_pc.csv")

(20530, 1218)


Combine methyl, rna, and labels

In [ ]:
#import pandas as pd

In [ ]:
methyl_highvar = pd.read_csv(path+"methyl_top50k_cg_anova.csv", index_col=0)
rna_pc = pd.read_csv(path+"rna_pc.csv", index_col=0)
labels = pd.read_csv(path+"brca_labels.csv", index_col=0)
rna_pc.shape, methyl_highvar.shape, labels.shape

((16479, 1218), (533, 50000), (837, 1))

In [ ]:
# fill missing with means
methyl = methyl_highvar.copy()
methyl = methyl.fillna(methyl.mean(), axis=0)
rna_pc = rna_pc.fillna(rna_pc.mean(axis=1), axis=0)

In [ ]:
# 1. load data
methyl_t = methyl.copy()
rna_t = rna_pc.T
labels_t = labels

# 2. align samples
common_samples = list(set(methyl_t.index) & set(rna_t.index) & set(labels_t.index))
print(f"total {len(common_samples)} samples aligned")

# 3. save data
methyl_aligned = methyl_t.loc[common_samples]
rna_aligned = rna_t.loc[common_samples]
labels_aligned = labels_t.loc[common_samples]

print(methyl_aligned.shape, rna_aligned.shape, labels_aligned.shape)

total 533 samples aligned
(533, 50000) (533, 16479) (533, 1)


Save aligned methyl, rna, label data

In [ ]:
methyl_aligned.to_csv(path+"methyl_aligned.csv")
rna_aligned.to_csv(path+"rna_aligned.csv")
labels_aligned.to_csv(path+"labels_aligned.csv")